In [1]:
! pip install kaggle


In [2]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download 'mahabuburrahman/celeba'

 99% 1.69G/1.70G [00:15<00:00, 55.1MB/s]
100% 1.70G/1.70G [00:16<00:00, 114MB/s] 


In [ ]:
! unzip 'celeba'

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [9]:
# Data preprocessing
train_data_dir = '/content/dataset/Train'
validation_data_dir = '/content/dataset/Validation'
test_data_dir = '/content/dataset/Test'
img_width, img_height = 150, 150
batch_size = 32

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [13]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)



Epoch 1/10
4375/4375 [==============================] - 262s 58ms/step - loss: 0.2691 - accuracy: 0.8779 - val_loss: 0.2182 - val_accuracy: 0.9101
Epoch 2/10
4375/4375 [==============================] - 267s 61ms/step - loss: 0.1317 - accuracy: 0.9467 - val_loss: 0.1764 - val_accuracy: 0.9269
Epoch 3/10
4375/4375 [==============================] - 233s 53ms/step - loss: 0.0989 - accuracy: 0.9603 - val_loss: 0.1687 - val_accuracy: 0.9319
Epoch 4/10
4375/4375 [==============================] - 216s 49ms/step - loss: 0.0752 - accuracy: 0.9698 - val_loss: 0.1805 - val_accuracy: 0.9358
Epoch 5/10
4375/4375 [==============================] - 217s 50ms/step - loss: 0.0590 - accuracy: 0.9766 - val_loss: 0.1880 - val_accuracy: 0.9368
Epoch 6/10
4375/4375 [==============================] - 217s 50ms/step - loss: 0.0440 - accuracy: 0.9828 - val_loss: 0.2356 - val_accuracy: 0.9394
Epoch 7/10
4375/4375 [==============================] - 219s 50ms/step - loss: 0.0355 - accuracy: 0.9863 - val_loss: 0

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('Test accuracy:', test_acc)

340/340 [==============================] - 13s 39ms/step - loss: 0.4606 - accuracy: 0.8918
Test accuracy: 0.8918198347091675


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [16]:
model.save('deepfake.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
img_path = '/content/Screenshot 2024-02-11 140333.png'
img = image.load_img(img_path, target_size=(img_width, img_height))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make prediction
prediction = model.predict(img_array)
if prediction[0] < 0.5:
    print("Predicted class: Real")
else:
    print("Predicted class: Fake")


1/1 [==============================] - 0s 313ms/step
Predicted class: Fake


In [18]:
# Load the image
img_path = '/content/Screenshot 2024-02-11 140426.png'
img = image.load_img(img_path, target_size=(img_width, img_height))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make prediction
prediction = model.predict(img_array)
if prediction[0] < 0.5:
    print("Predicted class: Real")
else:
    print("Predicted class: Fake")


1/1 [==============================] - 0s 26ms/step
Predicted class: Real
